Rough implementation of the classic arXiv paper, word2vec.

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
text = "the cat sat on the mat"

In [3]:
for i in text:
    print(i)
    break

t


In [4]:
!pip install nltk



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import nltk
nltk.download('brown')


[nltk_data] Downloading package brown to
[nltk_data]     /Users/kaushikdwivedi/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [6]:
from nltk.corpus import brown


In [7]:
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [8]:
text = "the cat sat on the mat"

In [9]:
brown.words()

['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]

In [10]:
for l in brown.words():
    print(l)
    break

The


In [11]:
len(brown.words())

1161192

In [12]:
words_small = list(brown.words())[:100000]

In [13]:
len(words_small)

100000

In [14]:
#we're starting our work with a 100,000 words for easier training
words = words_small

In [15]:
#building the vocabulary
#we first need to tokenise: remove punctuations, and turn capital letters into small


In [16]:
sample_text = ["The", "Cat's", "Dad", "sat", "on", "the", "mat"]

In [17]:
for word in sample_text:
    print(word.lower())

the
cat's
dad
sat
on
the
mat


In [18]:
import re

In [19]:
tokenizer_rndn = []
for word in sample_text:
    clean_parts = re.findall(r'\w+', word)
    tokenizer_rndn.extend(clean_parts)

In [20]:
tokenizer_rndn

['The', 'Cat', 's', 'Dad', 'sat', 'on', 'the', 'mat']

In [21]:
for word in tokenizer_rndn:
    word.lower()
    
    

In [22]:
tokenizer_rndn

['The', 'Cat', 's', 'Dad', 'sat', 'on', 'the', 'mat']

In [23]:
#words

In [24]:
import re

In [25]:
def tokenize(word_list):
    tokenized_words = []
    for word in word_list:
        
        word = word.lower()
        word = re.sub(r"'", "", word)
        
        clean_parts = re.findall(r"\w+", word)
        tokenized_words.extend(clean_parts)
    
    return tokenized_words

In [26]:
words = tokenize(words)

In [27]:
" ' " in words

False

In [28]:
"!" in words

False

In [29]:
"'" in words

False

In [30]:
"dont" in words

True

In [31]:
#done w the tokenizer, time to build the vocab
vocab = {}

In [32]:
len(words)

89563

In [33]:
j =0 


In [34]:
j

0

In [35]:
while(j<len(words)):
    vocab[j] = words[j]
    j=j+1

In [36]:
len(vocab)

89563

In [37]:
#since our dict is ready, we're now gonna create our emb table

In [85]:
emb_dim = 10
vocab_size = len(vocab)

In [86]:
emb_table = np.random.rand(vocab_size, emb_dim)

In [87]:
#emb_table

In [88]:
 #rough
'the' in vocab

False

In [89]:
0 in vocab

True

In [90]:
index_to_word = vocab

In [91]:
word_to_index = {word: idx for idx, word in vocab.items()}

In [92]:
#word_to_index

In [93]:
'the' in word_to_index

True

In [94]:
sentence = ["the", "cat", "is", "brown"]

In [95]:
#seems like our dataset doesnt have the word cat
#trying some other sentence
#trying to mimic some words

sentence = ["the", "election", "was", "grand"]

In [96]:
for i in sentence:
    print(word_to_index[i])

89559
75708
89024
80451


In [97]:
word_to_index['the']

89559

In [98]:
index_to_word[89559]

'the'

In [99]:
#rough
tokenized_corpus = []
for i in index_to_word:
    tokenized_corpus.append(i)

    

In [100]:
len(tokenized_corpus)

89563

In [101]:
class Pair:
    
    def __init__(self, center_idx, context_idx):
        self.center_idx = center_idx
        self.context_idx = context_idx
        
    def get_center(self):
        return self.center_idx
        
    def get_context(self):
        return self.context_idx
    
    def as_list(self):
        return [self.center_dx]
    
    def __str__(self):
        return f"({self.center_idx}, {self.context_idx})"

    def __repr__(self):
        return self.__str__()
        
    
        
        

In [102]:
class PairDataset:
    
    def __init__(self, pair_list):
        self.pairs = pair_list
    
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self,idx):
        return self.pairs[idx]
    
    def __str__(self):
        return f"PairDataset of {len(self)} pairs)"

In [103]:
s_w = 2

In [104]:
tokenized_corpus = list(range(89563))  # dummy index corpus


In [105]:
def create_pairs(list_of_corpurs):
    s_w = 2  # Define your window size (assumed 2 based on context)
    pair_list = []

    for i in range(len(tokenized_corpus)):
        # Handle the first word
        if i == 0:
            pair_list.append(Pair(i, i+1))
            pair_list.append(Pair(i, i+2))
            
        # Handle the second word
        elif i == 1:
            pair_list.append(Pair(i, i-1))
            pair_list.append(Pair(i, i+1))
            pair_list.append(Pair(i, i+2))
        
        # Handle the last word
        elif i == len(tokenized_corpus) - 1:
            pair_list.append(Pair(i, i-1))
            pair_list.append(Pair(i, i-2))
        
        # Handle the second-to-last word
        elif i == len(tokenized_corpus) - 2:
            pair_list.append(Pair(i, i+1))
            pair_list.append(Pair(i, i-1))
            pair_list.append(Pair(i, i-2))
            
        # Handle all other words (with a sliding window)
        else:
            # Words to the right
            if i + s_w < len(tokenized_corpus):
                pair_list.append(Pair(i, i+s_w))
            if i + s_w - 1 < len(tokenized_corpus):
                pair_list.append(Pair(i, i+s_w-1))
            
            # Words to the left
            if i - s_w >= 0:
                pair_list.append(Pair(i, i-s_w))
            if i - s_w + 1 >= 0:
                pair_list.append(Pair(i, i-s_w+1))
                
    return pair_list

In [106]:
pair_list = create_pairs(tokenized_corpus)

In [107]:
dataset = PairDataset(pair_list)

In [108]:
dataset[5]

(2, 4)

In [109]:
for pair in dataset:
    center_idx = pair.center_idx
    context_idx = pair.context_idx
    
    print(center_idx)
    break

0


In [110]:
w=np.random.rand(vocab_size,emb_dim)

In [111]:
import random

In [112]:
import numpy as np

def sigmoid(x):
    
    # Clip values to avoid overflow in exp
    x_clipped = np.clip(x, -20, 20)
    
    # Compute sigmoid
    return 1 / (1 + np.exp(-x_clipped))

In [113]:
list =[]
for i in range(100):
    list.append(i)

In [114]:
import random

In [115]:
j = random.choice(list)
j

80

In [124]:
class Word2Vec:
    
    def __init__(self, vocab_size, emb_dim, dataset, learning_rate=0.01, neg_samples = 5):
        
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim
        self.learning_rate = learning_rate
        self.neg_samples = neg_samples
        self.dataset = dataset
        
        self.W_in = np.random.rand(vocab_size, emb_dim)
        self.W_out = np.random.rand(vocab_size, emb_dim)
    
        
    def forward(self):
        print('Starting forward pass')
        loss = 0
        forward_cache = []

        positive_prob = []
        negative_prob = []
        all_neg_indices = []

        for i, pair in enumerate(self.dataset):
            if i % 1000 == 0:
                print(f"→ Forward pass: processing pair {i+1}/{len(self.dataset)}")

            center_idx = pair.center_idx
            context_idx = pair.context_idx

            # positive prob
            pos_prob = np.dot(self.W_in[center_idx], self.W_out[context_idx])
            pos_prob = sigmoid(pos_prob)
            positive_prob.append(pos_prob)

            # negative probs
            all_indices = list(range(self.vocab_size))
            all_indices.remove(context_idx)

            neg_probs_per_pair = []
            neg_indices = []

            for _ in range(self.neg_samples):
                neg_idx = random.choice(all_indices)
                neg_indices.append(neg_idx)
                neg_score = np.dot(self.W_in[center_idx], self.W_out[neg_idx])
                neg_prob = sigmoid(-neg_score)
                neg_probs_per_pair.append(neg_prob)

            negative_prob.append(neg_probs_per_pair)
            all_neg_indices.append(neg_indices)

        # convert to arrays
        positive_prob_array = np.array(positive_prob)
        negative_prob_array = np.array(negative_prob)

        pos_loss_total = -np.log(positive_prob_array)
        neg_loss_total = -np.sum(np.log(negative_prob_array), axis=1)

        total_loss_per_pair = pos_loss_total + neg_loss_total
        loss = np.mean(total_loss_per_pair)

        # store
        for idx, pair in enumerate(self.dataset):
            forward_cache.append({
                'center_idx': pair.center_idx,
                'context_idx': pair.context_idx,
                'neg_indices': all_neg_indices[idx],
                'pos_prob': positive_prob[idx],
                'neg_prob': negative_prob[idx]
            })

        return loss, forward_cache

        
    ####################################################################################################
    
    #backward pass
    def backward(self):
        
        print('starting backward pass')
        loss, forward_cache = self.forward()
        
        for item in forward_cache:
            center_idx = item['center_idx']
            context_idx = item['context_idx']
            neg_indices = item['neg_indices']
            pos_prob = item['pos_prob']
            neg_prob = item['neg_prob']
            
            #embeddings
            v_c = self.W_in[center_idx]
            v_o = self.W_out[context_idx]
            
            #grad wrt center from positive probs
            #initialise gradient for center
            grad_v_c =np.zeros_like(v_c)
            grad_v_c += (pos_prob-1)*v_o
            
            #grad wrt context
            grad_v_o = (pos_prob -1)*v_c
            
            #updating
            self.W_out[context_idx] -= self.learning_rate * grad_v_o
            
            #accumulate grad_v_C with neg samples
            for i, neg_prob_val in enumerate(neg_prob):
                neg_idx = neg_indices[i]
                v_n = self.W_out[neg_idx]
                grad_v_c +=(1-neg_prob_val)*v_n ##add to center gradient
                grad_v_n = (1-neg_prob_val)*v_c #gradient for this sample
                self.W_out[neg_idx] -= self.learning_rate*grad_v_n
                
            #update the center embedding
            self.W_in[center_idx] -= self.learning_rate*grad_v_c
                
        return loss
    
    def train(self, epochs):
        for epoch in range(epochs):
            loss = self.backward()
            print(f"Epoch {epoch+1}, Loss:{loss:.4f}")
            

In [125]:
vocab_size

89563

In [126]:
emb_dim

10

In [127]:
dataset

In [128]:
del list

NameError: name 'list' is not defined

In [82]:
model = Word2Vec(vocab_size, emb_dim, dataset.pairs)


In [83]:
model.train(epochs=1)

starting backward pass
Starting forward pass
→ Forward pass: processing pair 1/358246
→ Forward pass: processing pair 1001/358246
→ Forward pass: processing pair 2001/358246
→ Forward pass: processing pair 3001/358246
→ Forward pass: processing pair 4001/358246
→ Forward pass: processing pair 5001/358246
→ Forward pass: processing pair 6001/358246
→ Forward pass: processing pair 7001/358246
→ Forward pass: processing pair 8001/358246
→ Forward pass: processing pair 9001/358246
→ Forward pass: processing pair 10001/358246
→ Forward pass: processing pair 11001/358246
→ Forward pass: processing pair 12001/358246
→ Forward pass: processing pair 13001/358246
→ Forward pass: processing pair 14001/358246
→ Forward pass: processing pair 15001/358246
→ Forward pass: processing pair 16001/358246
→ Forward pass: processing pair 17001/358246
→ Forward pass: processing pair 18001/358246
→ Forward pass: processing pair 19001/358246
→ Forward pass: processing pair 20001/358246
→ Forward pass: process

In [129]:
model_2 = Word2Vec(vocab_size, emb_dim, dataset.pairs)

In [132]:
model_2.train(epochs)

starting backward pass
Starting forward pass
→ Forward pass: processing pair 1/358246
→ Forward pass: processing pair 1001/358246
→ Forward pass: processing pair 2001/358246
→ Forward pass: processing pair 3001/358246
→ Forward pass: processing pair 4001/358246
→ Forward pass: processing pair 5001/358246
→ Forward pass: processing pair 6001/358246
→ Forward pass: processing pair 7001/358246
→ Forward pass: processing pair 8001/358246
→ Forward pass: processing pair 9001/358246
→ Forward pass: processing pair 10001/358246
→ Forward pass: processing pair 11001/358246
→ Forward pass: processing pair 12001/358246
→ Forward pass: processing pair 13001/358246
→ Forward pass: processing pair 14001/358246
→ Forward pass: processing pair 15001/358246
→ Forward pass: processing pair 16001/358246
→ Forward pass: processing pair 17001/358246
→ Forward pass: processing pair 18001/358246
→ Forward pass: processing pair 19001/358246
→ Forward pass: processing pair 20001/358246
→ Forward pass: process

In [133]:
import numpy as np

def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def word_analogy(word1, word2, word3, model, word2idx, idx2word, top_n=5):
    v1 = model.W_in[word2idx[word1]]
    v2 = model.W_in[word2idx[word2]]
    v3 = model.W_in[word2idx[word3]]

    target_vec = v1 - v2 + v3

    sims = []
    for i in range(model.vocab_size):
        word = idx2word[i]
        vec = model.W_in[i]
        sim = cosine_sim(target_vec, vec)
        sims.append((word, sim))

    sims = sorted(sims, key=lambda x: -x[1])
    return sims[:top_n]


In [136]:
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def word_analogy(w1, w2, w3, model, word_to_index, index_to_word, top_n=5):
    v1 = model.W_in[word_to_index[w1]]
    v2 = model.W_in[word_to_index[w2]]
    v3 = model.W_in[word_to_index[w3]]

    target_vec = v1 - v2 + v3

    similarities = []
    for i in range(model.vocab_size):
        candidate_vec = model.W_in[i]
        sim = cosine_sim(target_vec, candidate_vec)
        similarities.append((index_to_word[i], sim))

    similarities = sorted(similarities, key=lambda x: -x[1])
    return similarities[:top_n]


In [137]:
word_analogy("king", "man", "woman", model, word_to_index, index_to_word)


[('king', np.float64(0.802859473455564)),
 ('woman', np.float64(0.7914603173289692)),
 ('to', np.float64(0.7021909560508013)),
 ('november', np.float64(0.6986139523926287)),
 ('possible', np.float64(0.6905423686069705))]

In [138]:
word_analogy("paris", "france", "germany", model, word_to_index, index_to_word)


[('paris', np.float64(0.7625240431494841)),
 ('germany', np.float64(0.747907996591123)),
 ('nearby', np.float64(0.6947087358874876)),
 ('table', np.float64(0.6873449937369923)),
 ('physically', np.float64(0.6824033998676439))]

In [139]:
word_analogy("doctor", "man", "woman", model, word_to_index, index_to_word)


[('woman', np.float64(0.796290867205041)),
 ('doctor', np.float64(0.7950196165753333)),
 ('manner', np.float64(0.7296810365592593)),
 ('to', np.float64(0.7121098421837767)),
 ('very', np.float64(0.700102654237117))]

In [142]:
tests = [
    ("king", "man", "woman"),
    ("paris", "france", "italy"),
    ("berlin", "germany", "france"),
    ("doctor", "man", "woman"),
    ("uncle", "man", "woman"),
    ("he", "man", "woman"),
    ("sun", "day", "night"),
    ("summer", "hot", "cold"),
    ("run", "running", "swimming"),
    ("big", "bigger", "smaller")
]

for w1, w2, w3 in tests:
    try:
        result = word_analogy(w1, w2, w3, model, word_to_index, index_to_word)
        print(f"\n{w1} - {w2} + {w3} :")
        for word, sim in result:
            print(f"  {word:10s}  ||  sim = {sim:.4f}")
    except KeyError as e:
        print(f"\n Skipped {w1}-{w2}+{w3} — missing word: {e}")



king - man + woman :
  king        ||  sim = 0.8029
  woman       ||  sim = 0.7915
  to          ||  sim = 0.7022
  november    ||  sim = 0.6986
  possible    ||  sim = 0.6905

paris - france + italy :
  paris       ||  sim = 0.7480
  italy       ||  sim = 0.7412
  and         ||  sim = 0.6545
  mails       ||  sim = 0.6464
  which       ||  sim = 0.6455

berlin - germany + france :
  berlin      ||  sim = 0.8022
  france      ||  sim = 0.7925
  beneath     ||  sim = 0.7446
  of          ||  sim = 0.7356
  their       ||  sim = 0.7323

doctor - man + woman :
  woman       ||  sim = 0.7963
  doctor      ||  sim = 0.7950
  manner      ||  sim = 0.7297
  to          ||  sim = 0.7121
  very        ||  sim = 0.7001

uncle - man + woman :
  woman       ||  sim = 0.8205
  uncle       ||  sim = 0.7506
  scholar     ||  sim = 0.7302
  and         ||  sim = 0.7132
  kennedy     ||  sim = 0.7131

he - man + woman :
  woman       ||  sim = 0.8293
  he          ||  sim = 0.7831
  a           ||  s

In [143]:
np.savez("word2vec_model.npz", W_in=model.W_in, W_out=model.W_out)
